## Note on `git` and `ws3` embedded as submodule.

I am tracking this notebook-based project on GitHub, and have embedded the ws3 as a submodule. The following code pulls ws3 code from the cloud.

In [4]:
!git submodule init
!git submodule update

The following is a hack to force this notebook to use the local `ws3` submodule code.

In [5]:
import os
import sys
module_path = os.path.abspath(os.path.join('ws3'))
if module_path not in sys.path:
    sys.path.append(module_path)
sys.path

['/media/data/project/gparadis/model_prototype_0',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/opt/jupyterhub/lib/python3.10/site-packages',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/media/data/project/gparadis/model_prototype_0/ws3']

# Compile Woodstock-style input files

This notebook was created by Greg Paradis.

We implement prototype forest estate model, which will form the basis of the forest modelling Jimmy Ke will do in the context of his thesis project.

The model is implemented using `ws3`. We use `Woodstock`-style text input files to compile the basic input data for our `ws3` model (i.e., stratification schema, initial inventory, yield curves, treatments, transitions). Some of these text files are hand-coded, while others are (at least partially) generated from Python code in this notebook. 

## Preamble

Import modules, define classes and functions and constants, etc.

In [1]:
import pandas as pd
import geopandas as gpd
import ws3.forest, ws3.core

/opt/jupyterhub/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


The default `ws3.forest.GreedyAreaSelector` is broken. This is basically just a copy of that but patched. Need to fix this ASAP and push a patch up to the GitHub repo.

In [2]:
class GreedyAreaSelector:
    """
    Default AreaSelector implementation. Selects areas for treatment from oldest age classes.
    """
    def __init__(self, parent):
        self.parent = parent

    def operate(self, period, acode, target_area, mask=None,
                commit_actions=True, verbose=False):
        """
        Greedily operate on oldest operable age classes.
        Returns missing area (i.e., difference between target and operated areas).
        """
        key = lambda item: max(item[1])
        odt = sorted(list(self.parent.operable_dtypes(acode, period, mask).items()), key=key)
        print(' entering selector.operate()', len(odt), 'operable dtypes')
        while target_area > 0 and odt:
            while target_area > 0 and odt:
                popped = odt.pop()
                try:
                    dtk, ages = popped #odt.pop()
                except:
                    print(odt)
                    print(popped)
                    raise
                age = sorted(ages)[-1]
                oa = self.parent.dtypes[dtk].operable_area(acode, period, age)
                if not oa: continue # nothing to operate
                area = min(oa, target_area)
                target_area -= area
                if area < 0:
                    print('negative area', area, oa, target_area, acode, period, age)
                    assert False
                if verbose:
                    print(' selector found area', [' '.join(dtk)], acode, period, age, area)
                self.parent.apply_action(dtk, acode, period, age, area, 
                                         fuzzy_age=False, recourse_enabled=False, 
                                         compile_c_ycomps=True, verbose=verbose)
            odt = sorted(list(self.parent.operable_dtypes(acode, period, mask).items()), key=key)
        self.parent.commit_actions(period, repair_future_actions=True)
        if verbose:
            print('GreedyAreaSelector.operate done (remaining target_area: %0.1f)' % target_area)
        return target_area

Define a function that implements a greedy (oldest-first) priority queue heuristic harvest scheduling heuristic (including auto-parametrizing the periodic harvest area target with some bootstrap analysis of the yield curves we built into the model). Just for quick and dirty testing purposes (less finnicky that formulating and solving an optimization problem, and also does not require a working `gurobi` solver). 

In [37]:
# copy function definition for debugging
def schedule_harvest_areacontrol(fm, period=1, acode='cc', util=0.85, 
                                 target_masks=None, target_areas=None, target_scalefactors=None,
                                 mask_area_thresh=0.,
                                 verbose=0):
    #fm.reset_actions()
    if not target_areas:
        if not target_masks: # default to AU-wise THLB 
            au_vals = []
            au_agg = []
            for au in fm.theme_basecodes(2):
                mask = '? 1 %s ? ?' % au
                masked_area = fm.inventory(0, mask=mask)
                #print(mask, masked_area)
                if masked_area > mask_area_thresh:
                    #print(masked_area, mask_area_thresh)
                    au_vals.append(au)
                else:
                    au_agg.append(au)
                    if verbose > 0:
                        print('adding to au_agg', mask, masked_area)
                #print(au_vals)
            if au_agg:
                fm._themes[2]['areacontrol_au_agg'] = au_agg 
                if fm.inventory(0, mask='? ? areacontrol_au_agg ? ?') > mask_area_thresh:
                    au_vals.append('areacontrol_au_agg')
            target_masks = ['? 1 %s ? ?' % au for au in au_vals]
            #print(au_vals)
            #print(target_masks)
        print(target_masks, au_vals)
        #assert False
        target_areas = []
        for i, mask in enumerate(target_masks): # compute area-weighted mean CMAI age for each masked DT set
            print(mask)
            masked_area = fm.inventory(0, mask=mask, verbose=verbose)
            if not masked_area: continue
            r = sum((fm.dtypes[dtk].ycomp('volume').mai().ytp().lookup(0) * fm.dtypes[dtk].area(0)) for dtk in fm.unmask(mask))
            r /= masked_area
            #awr = []
            #dtype_keys = fm.unmask(mask)
            #for dtk in dtype_keys:
            #    dt = fm.dtypes[dtk]
            #    awr.append(dt.ycomp('totvol').mai().ytp().lookup(0) * dt.area(0))
            #r = sum(awr)  / masked_area
            asf = 1. if not target_scalefactors else target_scalefactors[i]  
            print(i)
            print(mask)
            print(masked_area)
            ta = (1/r) * fm.period_length * masked_area * asf
            target_areas.append(ta)
    for mask, target_area in zip(target_masks, target_areas):
        if verbose > 0:
            print('calling areaselector', period, acode, target_area, mask)
        fm.areaselector.operate(period, acode, target_area, mask=mask, verbose=verbose)
    sch = fm.compile_schedule()
    return sch

Define some constrants and such.

In [4]:
data_path = 'data/input'
fert_codes = ['00',
              '11',
              '12',
              '13',
              '14',
              '15',
              '16',
              '17',
              '21',
              '22',
              '23',
              '24',
              '25',
              '26',
              '31',
              '32',
              '33',
              '34',
              '35']
period_length = 10
max_age = 350
base_year = 2020
horizon = 10
curve_table = pd.read_csv('%s/curve_table.csv' % data_path)
model_name = 'tsa24_clipped'

# Compile LANDSCAPE section

Hand coded to `data_path/model_name.lan`.

# Compile AREAS section

In [46]:
stands = gpd.read_file('%s/shp/tsa24_clipped.shp/stands.shp' % data_path)

Need to rebuild the THLB attribute in the stand inventory table. Current implementation is inconsistent with yield VDYP/TIPSY yield curve (and AU definition) modelling assumptions. 

In a nutshell, AUs either have only a VDYP yield curve or both VDYP and TISPY yield curves (if considered potentially operable). We need to set the THLB attribute to 0 for stands in AUs with have only a VDYP curve (i.e., where `au_table.unmanaged_curve_id == au_table.managed_curve_id`), and 1 otherwise. 

Otherwise, we would get weird cases where we can harvest a stand but there is not TIPSY curve defined for second-growth stand conditions.

In [47]:
au_table = pd.read_csv('%s/au_table.csv' % data_path).set_index('au_id')
curve_table = pd.read_csv('%s/curve_table.csv' % data_path)
curve_points_table = pd.read_csv('%s/curve_points_table.csv' % data_path).set_index('curve_id')

In [48]:
au_table = au_table.query('tsa == 24')

In [49]:
au_table['thlb'] = au_table.apply(lambda row: 0 if row.unmanaged_curve_id == row.managed_curve_id else 1, axis=1)

In [50]:
stands.theme1 = stands.apply(lambda row: au_table.loc[row.theme2].thlb, axis=1)

Copy `curve1` to `theme4` so we can track yield curve transitions independently from AU.

In [51]:
stands['theme4'] = stands.curve1

Set up themes.

In [52]:
theme_cols=['theme0', # TSA 
            'theme1', # THLB
            'theme2', # AU
            'theme3', # leading species code
            'theme4'] # yield curve ID

In [53]:
gstands = stands.groupby(theme_cols+['age'])

In [93]:
lines = []
for name, group in gstands:
    dtk, age, area = tuple(map(lambda x: str(x), name[:-1])), int(name[-1]), group['area'].sum()
    if dtk[1] == '0': continue
    if dtk[4][2] == '0':
        t4 = list(dtk[4])
        t4[2] = '2'
        t4 = ''.join(t4)
        dtk = (dtk[0], dtk[1], dtk[2], dtk[3], t4)
        print(dtk)
    #print(name, group)
    #print(dtk, age, area)
    #print('*A', ' '.join(v for v in dtk), age, area)
    lines.append('*A %s %i %0.1f\n' % (' '.join(v for v in dtk), age, area))
    
filename = '%s/%s.are' % (data_path, model_name)
print(filename)
with open(filename, 'w') as snk:
    snk.writelines(lines)

('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2401002', '204', '2421002')
('tsa24_clipped', '1', '2402000', '100', '2422000')
('tsa24_clipped', '1', '2402002', '204', '2422002')
('tsa24_clipped', '1', '2402002', '204', '2422002')
('tsa24_clipped', '1', '2402002', '204', '2422002')
('tsa24_clipped', '1', '2402002', '204', '2422002')
('tsa24_clip

# Compile YIELDS section

In [94]:
lines = []
# for fert
for fert in fert_codes:
    au_table = pd.read_csv('%s/au_table_%s.csv' % (data_path, fert)).set_index('au_id')
    au_table = au_table.query('tsa == 24')
    au_table['thlb'] = au_table.apply(lambda row: 0 if row.unmanaged_curve_id == row.managed_curve_id else 1, axis=1)
    curve_points_table = pd.read_csv('%s/curve_points_table_updated_%s.csv' % (data_path, fert)).set_index('curve_id')
    for au_id, au_row in au_table.iterrows():
        if au_row.thlb == 0:
            fert_curve_id = au_row.fert_curve_id #apply curve id 
            managed_curve_id = au_row.managed_curve_id
            curve_id = au_row.managed_curve_id 
            mask = ('?', '?', str(au_id), '?', str(managed_curve_id))
            continue # (hack) omit unmanaged, i.e., NTHLB, AUs from the model
        else:
            fert_curve_id = au_row.fert_curve_id #apply curve id 
            managed_curve_id = au_row.managed_curve_id
            curve_id = au_row.fert_curve_id
            mask = ('?', '?', str(au_id), '?', str(fert_curve_id))
        lines.append('*Y %s\n' % ' '.join(v for v in mask))
        for i, col in enumerate(curve_points_table.columns):
            if i == 0:
                continue 
            yname = col
            points = [(r.age, r[col]) for _, r in curve_points_table.loc[curve_id].iterrows() if not r.age % period_length and r.age <= max_age]
            is_volume = True if yname == 'volume' else False
            c = ws3.core.Curve(yname, points=points, type='a', is_volume=is_volume, xmax=max_age, period_length=period_length)
            if not au_row.thlb: continue
            lines.append('%s 1 %s\n' % (yname, ' '.join(str(int(c[x])) for x in range(0, 300, 1))))
filename = '%s/%s.yld' % (data_path, model_name)
print(filename)
with open(filename, 'w') as snk:
    snk.writelines(lines)

data/input/tsa24_clipped.yld


# Compile ACTIONS section

In [122]:
lines = []

lines.append('*ACTION cc Y clearcut harvest\n')
lines.append('*OPERABLE cc\n')
lines.append('? 1 ? ? ? _AGE >= 40 AND _AGE <= 999\n')

for i in range(1,8):
    lines.append('*ACTION f1%i N fertilize once at age %i0\n' % (i,i))
    lines.append('*OPERABLE f1%i\n' % i)
    lines.append('? 1 ? ? unfertilized _AGE >= %i0 AND _AGE <= %i0\n' % (i,i))
for i in range(1,7):
    lines.append('*ACTION f2%i N fertilize twice at age %i0 %i0\n' % (i,i,i+1))
    lines.append('*OPERABLE f2%i\n' % i)
    lines.append('? 1 ? ? unfertilized _AGE >= %i0 AND _AGE <= %i0\n' % (i,i))
for i in range(1,6):
    lines.append('*ACTION f3%i N fertilize thrice at age %i0 %i0 %i0\n' % (i,i,i+1,i+2))
    lines.append('*OPERABLE f3%i\n' % i)
    lines.append('? 1 ? ? unfertilized _AGE >= %i0 AND _AGE <= %i0\n' % (i,i))
    
filename = '%s/%s.act' % (data_path, model_name)
print(filename)
with open(filename, 'w') as snk:
    snk.writelines(lines)

data/input/tsa24_clipped.act


# Compile TRANSITIONS section

In [96]:
lines = []

acode = 'cc'
lines.append('*CASE %s\n' % acode)
for au_id, au_row in au_table.iterrows():
    if not au_row.thlb: continue
    curve_id = au_row.managed_curve_id
    target_curve_id = au_row.managed_curve_id
    smask = ' '.join(('?', '?', str(au_id), '?', '?'))
    tmask = ' '.join(('?', '?', '?', '?', str(target_curve_id)))
    lines.append('*SOURCE %s\n' % smask)
    lines.append('*TARGET %s 100\n' % tmask)
    
fert_once = list(range(11, 18))
fert_twice = list(range(21,27))
fert_thrice = list(range(31,36))
fert_option_all = fert_once + fert_twice + fert_thrice
for i in fert_option_all:
    acode = 'f%i' % i
    lines.append('*CASE %s\n' % acode)
    for au_id, au_row in au_table.iterrows():
        if not au_row.thlb: continue
        curve_id = au_row.managed_curve_id
        target_curve_id = curve_id + i * 10
        smask = ' '.join(('?', '?', str(au_id), '?', '?'))
        tmask = ' '.join(('?', '?', '?', '?', str(target_curve_id)))
        lines.append('*SOURCE %s\n' % smask)
        lines.append('*TARGET %s 100\n' % tmask)
    
filename = '%s/%s.trn' % (data_path, model_name)
print(filename)
with open(filename, 'w') as snk:
    snk.writelines(lines)

data/input/tsa24_clipped.trn


# Create a new `ForestModel` instance (quick test)

Create a model from the input files, and try to run the model. Note that this level of testing is _not_ sufficient to definitively validate the model... this is just a quick and dirty "stink test" (i.e., if this fails there there is something wrong, but if this does not fail then there might still be something wrong lurking in the shadows). 

In [123]:
fm = ws3.forest.ForestModel(model_name=model_name,
                            model_path=data_path,
                            base_year=base_year,
                            horizon=horizon,
                            period_length=period_length,
                            max_age=max_age)

In [124]:
fm.import_landscape_section()

In [125]:
fm.import_areas_section()

0

In [126]:
fm.import_yields_section()

('?', '?', '?', '?', '?') 5
('?', '?', '2402000', '?', '2422000') 1
('?', '?', '2403000', '?', '2423000') 1
('?', '?', '2403001', '?', '2423001') 0
('?', '?', '2401002', '?', '2421002') 1
('?', '?', '2402002', '?', '2422002') 1
('?', '?', '2403002', '?', '2423002') 1
('?', '?', '2402003', '?', '2422003') 0
('?', '?', '2403003', '?', '2423003') 0
('?', '?', '2402004', '?', '2422004') 0
('?', '?', '2403004', '?', '2423004') 0
('?', '?', '2401007', '?', '2421007') 0
('?', '?', '2402007', '?', '2422007') 0
('?', '?', '2403007', '?', '2423007') 0
('?', '?', '2402000', '?', '2422110') 0
('?', '?', '2403000', '?', '2423110') 0
('?', '?', '2403001', '?', '2423111') 0
('?', '?', '2401002', '?', '2421112') 0
('?', '?', '2402002', '?', '2422112') 0
('?', '?', '2403002', '?', '2423112') 0
('?', '?', '2402003', '?', '2422113') 0
('?', '?', '2403003', '?', '2423113') 0
('?', '?', '2402004', '?', '2422114') 0
('?', '?', '2403004', '?', '2423114') 0
('?', '?', '2401007', '?', '2421117') 0
('?', '?', '

In [127]:
fm.import_actions_section()

In [128]:
fm.import_transitions_section()

In [129]:
fm.compile_actions()

In [130]:
fm.initialize_areas()
fm.reset_actions()

In [131]:
fm.areaselector = GreedyAreaSelector(fm)

In [132]:
#schedule_harvest_areacontrol(fm, period=1, verbose=1)

In [133]:
for period in fm.periods:
    schedule_harvest_areacontrol(fm, period=period, verbose=1)

adding to au_agg ? 1 2401004 ? ? 0.0
adding to au_agg ? 1 2401005 ? ? 0.0
adding to au_agg ? 1 2403005 ? ? 0.0
adding to au_agg ? 1 2401000 ? ? 0.0
adding to au_agg ? 1 2402007 ? ? 0.0
adding to au_agg ? 1 2402006 ? ? 0.0
adding to au_agg ? 1 2402003 ? ? 0.0
adding to au_agg ? 1 2403004 ? ? 0.0
adding to au_agg ? 1 2403007 ? ? 0.0
adding to au_agg ? 1 2401001 ? ? 0.0
adding to au_agg ? 1 2403006 ? ? 0.0
adding to au_agg ? 1 2402004 ? ? 0.0
adding to au_agg ? 1 2403003 ? ? 0.0
adding to au_agg ? 1 2401007 ? ? 0.0
adding to au_agg ? 1 2401003 ? ? 0.0
adding to au_agg ? 1 2401006 ? ? 0.0
adding to au_agg ? 1 2403001 ? ? 0.0
adding to au_agg ? 1 2402001 ? ? 0.0
adding to au_agg ? 1 2402005 ? ? 0.0
['? 1 2402000 ? ?', '? 1 2401002 ? ?', '? 1 2403000 ? ?', '? 1 2402002 ? ?', '? 1 2403002 ? ?'] ['2402000', '2401002', '2403000', '2402002', '2403002']
? 1 2402000 ? ?
0
? 1 2402000 ? ?
0.6
? 1 2401002 ? ?
1
? 1 2401002 ? ?
964.0000000000001
? 1 2403000 ? ?
2
? 1 2403000 ? ?
14.8
? 1 2402002 ? ?


In [134]:
pareas = [fm.compile_product(period, '1.') for period in fm.periods]

In [135]:
pvols = [fm.compile_product(period, 'volume') for period in fm.periods]

In [136]:
df = pd.DataFrame({'period':fm.periods, 'areas':pareas, 'vols':pvols})
df

,period,areas,vols
0,1,94.918575,22512.770978
1,2,92.718575,21083.512526
2,3,92.718575,20131.445809
3,4,92.718575,19083.192172
4,5,92.718575,17127.721953
5,6,92.718575,15643.384988
6,7,92.718575,15319.406701
7,8,92.718575,14203.266722
8,9,92.718575,14034.655902
9,10,92.718575,14233.099653
